# **TP Intelligence Artificielle - SAT et CSP**

# **Partie 0 : Visualisation**

Voici une fonction pour visualiser les sudokus.

In [1]:
from IPython.display import HTML

def visualize_sudoku(grid):
    html = "<table>"
    for i in range(9):
        html += "<tr>"
        for j in range(9):
            value = grid[i][j]
            cell_style = "width: 30px; height: 30px; text-align: center; font-size: 20px; border: 1px solid black;"
            if i % 3 == 0 and i != 0:  # Horizontal lines
                cell_style += "border-top: 5px solid black;"
            if j % 3 == 0 and j != 0:  # Vertical lines
                cell_style += "border-left: 5px solid black;"

            if value == 0:
                html += f"<td style='background-color: white; {cell_style}'> </td>"  # Case vide
            else:
                html += f"<td style='background-color: rgb(226, 114, 91); {cell_style}'>{value}</td>"
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

### **Exercice 1**
Écrivez un Sudoku sous forme d’une liste de listes (9 listes de 9 élements chacune où la valeur 0 represente une caise vide), puis utilisez la fonction visualize_sudoku pour l’afficher.

In [2]:

grid = [
    [0, 0, 0, 7, 0, 0, 0, 0, 0],
    [0, 0, 7, 0, 5, 1, 0, 0, 0],
    [9, 0, 5, 0, 3, 8, 0, 0, 7],
    [0, 9, 0, 5, 0, 7, 0, 1, 0],
    [4, 0, 1, 0, 0, 0, 3, 6, 0],
    [2, 0, 0, 1, 0, 0, 0, 7, 0],
    [0, 0, 4, 0, 8, 0, 0, 0, 3],
    [0, 3, 0, 0, 7, 0, 0, 0, 2],
    [0, 0, 6, 0, 0, 0, 0, 0, 0]
]
visualize_sudoku(grid)

,,,7,,,,,
,,7,,5,1,,,
9,,5,,3,8,,,7
,9,,5,,7,,1,
4,,1,,,,3,6,
2,,,1,,,,7,
,,4,,8,,,,3
,3,,,7,,,,2
,,6,,,,,,


# **Partie 1 : Modélisation**

Nous allons modéliser le Sudoku comme un problème SAT et CSP, puis utiliser les méthodes propres à chaque modélisation pour la résolution.
Pour cela, nous utiliserons deux bibliothèques : **python-sat** et **python-constraint**.

Exécutez le bloc de code suivant pour les installer.
Pour les installer sur vos PC, il suffit d’exécuter les mêmes lignes (sans le '!' au début) dans une console de commandes (CMD).



In [3]:
# Installation des libraries
# pip install python-sat
# pip install python-constraint

## **1.1 Modélisation SAT**

Pour modéliser le Sudoku comme un problème SAT, nous utiliserons la méthode CNF de la bibliothèque pysat.formula.
CNF permet de définir les clauses logiques représentant les contraintes du problème.
Pour les variables, nous utiliserons un dictionnaire.

In [4]:
from pysat.formula import CNF

class Sudoku_SAT:
    def __init__(self, grid):
        self.grid = grid
        self.cnf = CNF()  # Crée un objet CNF pour stocker les clauses
        self.var_map = {}  # Dictionnaire pour mapper (ligne, colonne, valeur) à un ID de variable SAT

        # 1. Créer les variables SAT
        var_count = 1
        for i in range(9):
            for j in range(9):
                for k in range(1, 10):
                    self.var_map[(i, j, k)] = var_count
                    var_count += 1

        # 2. Créer les clauses SAT

        # a. Chaque case doit contenir au moins une valeur
        for i in range(9):
            for j in range(9):
                clause = [self.var_map[(i, j, k)] for k in range(1, 10)]
                self.cnf.append(clause)

        # b. Chaque case ne peut contenir qu'une seule valeur
        for i in range(9):
            for j in range(9):
                for k1 in range(1, 10):
                    for k2 in range(k1 + 1, 10):
                        clause = [-self.var_map[(i, j, k1)], -self.var_map[(i, j, k2)]]
                        self.cnf.append(clause)

        # c. Chaque ligne doit contenir tous les chiffres de 1 à 9
        for i in range(9):
            for k in range(1, 10):
                clause = [self.var_map[(i, j, k)] for j in range(9)]
                self.cnf.append(clause)

        # d. Chaque colonne doit contenir tous les chiffres de 1 à 9
        for j in range(9):
            for k in range(1, 10):
                clause = [self.var_map[(i, j, k)] for i in range(9)]
                self.cnf.append(clause)

        # e. Chaque sous-grille 3x3 doit contenir tous les chiffres de 1 à 9
        for subgrid_row in range(3):
            for subgrid_col in range(3):
                for k in range(1, 10):
                    clause = [self.var_map[(subgrid_row * 3 + i, subgrid_col * 3 + j, k)]
                              for i in range(3) for j in range(3)]
                    self.cnf.append(clause)

        # f. Contraintes initiales de la grille
        for i in range(9):
            for j in range(9):
                if self.grid[i][j] != 0:
                    self.cnf.append([self.var_map[(i, j, self.grid[i][j])]])

### **Exercice 2**
Créez une fonction qui prend une grille de Sudoku en entrée et utilise le solveur Glucose3 de la bibliothèque pysat.solver pour résoudre le jeu (pensez à bien importer Glucosa3). Votre fonction doit :

1. Créer un objet sudoku_sat pour modéliser le problème.
2. Ajouter au solveur les contraintes du Sudoku sous forme de clauses logiques.
3. Vérifier si le Sudoku est résoluble en utilisant le solveur SAT.
4. Trouver une solution si le Sudoku est résoluble.
5. Interpréter la solution retournée par le solveur pour remplir la grille :
  * La solution sera une liste de taille 9 × 9 × 10, contenant des valeurs positives et négatives.
  * Une valeur positive signifie que la variable (i, j, k) est True, ce qui indique que la case (i, j) doit contenir la valeur k.

In [5]:
from pysat.solvers import Glucose3

def solve_sudoku_sat(grid):
    """
    使用 SAT 求解器 (Glucose3) 解决数独问题。
    """
    # 1. 创建 sudoku_sat 对象来建模问题 (生成 CNF 子句)
    model = Sudoku_SAT(grid)
    
    # 2. 初始化求解器
    # 使用 'with' 语句可以确保求解器在使用后正确关闭释放资源
    with Glucose3() as solver:
        # 3. 将数独的约束 (CNF 子句) 添加到求解器中
        solver.append_formula(model.cnf.clauses)
        
        # 4. 检查数独是否可解
        if solver.solve():
            # 如果可解，获取模型 (即让所有子句为真的变量赋值列表)
            solution = solver.get_model()
            solution_set = set(solution)  # 转换为集合以加快查找速度
            
            # 5. 解释解决方案并填充网格
            # 遍历我们在 Sudoku_SAT 中创建的映射: (行, 列, 值) -> var_id
            for (i, j, k), var_id in model.var_map.items():
                # 如果 var_id 在解决方案中 (即该变量为 True)
                # 意味着在第 i 行第 j 列填入数字 k
                if var_id in solution_set:
                    grid[i][j] = k
            
            return grid
        else:
            print("Pas de solution trouvée (无解)")
            return None

### **Exercice 3**
Utilisez votre fonction pour resoudre le sudoku ci-dessous:

    [0, 0, 0, 0, 7, 0, 0, 0, 0]
    [6, 0, 0, 1, 9, 5, 0, 0, 0]
    [0, 9, 8, 0, 0, 0, 0, 6, 0]
    [8, 0, 0, 0, 6, 0, 0, 0, 3]
    [4, 0, 0, 8, 0, 3, 0, 0, 1]
    [7, 0, 0, 0, 2, 0, 0, 0, 6]
    [0, 6, 0, 0, 0, 0, 2, 8, 0]
    [0, 0, 0, 4, 1, 9, 0, 0, 5]
    [0, 0, 0, 0, 8, 0, 0, 7, 9]

In [6]:
grid = [
    [0, 0, 0, 7, 0, 0, 0, 0, 0],
    [0, 0, 7, 0, 5, 1, 0, 0, 0],
    [9, 0, 5, 0, 3, 8, 0, 0, 7],
    [0, 9, 0, 5, 0, 7, 0, 1, 0],
    [4, 0, 1, 0, 0, 0, 3, 6, 0],
    [2, 0, 0, 1, 0, 0, 0, 7, 0],
    [0, 0, 4, 0, 8, 0, 0, 0, 3],
    [0, 3, 0, 0, 7, 0, 0, 0, 2],
    [0, 0, 6, 0, 0, 0, 0, 0, 0],
]

res = solve_sudoku_sat(grid)
if res :
    visualize_sudoku(res)
else :
    print("aucune solution")


8,6,2,7,9,4,5,3,1
3,4,7,2,5,1,8,9,6
9,1,5,6,3,8,4,2,7
6,9,3,5,4,7,2,1,8
4,7,1,8,2,9,3,6,5
2,5,8,1,6,3,9,7,4
7,2,4,9,8,6,1,5,3
1,3,9,4,7,5,6,8,2
5,8,6,3,1,2,7,4,9


## **1.2 Modélisation CSP**

### **Exercice 4**

Créez une fonction solve_sudoku_csp qui prend une grille de Sudoku en entrée, la modélise comme un problème CSP et la résout. Votre fonction doit :
1. Créer une instance du problème CSP
  * Utilisez la classe Problem de la bibliothèque python-constraint pour représenter le Sudoku.
2. Définir les variables du problème
  * Utilisez la méthode addVariable pour ajouter les variables au problème, sachant que :
    * Chaque case de la grille est une variable identifiée par un tuple (ligne, colonne).
    * Le domaine de chaque variable correspond aux valeurs possibles (de 1 à 9).
    * Si une case contient déjà un chiffre (non nul), son domaine est restreint à cette valeur unique.
3. Définir les contraintes du Sudoku (les valeurs doivent être uniques dans chaque ligne, colonne et région 3×3). Pour cela, utilisez la méthode addConstraint avec AllDifferentConstraint() pour assurer ces contraintes.
4. Résoudre le problème. Pour cela utilisez la méthode getSolutions() pour obtenir toutes les solutions possibles.
5. Gérer les solutions
  * Si des solutions existent, les stocker dans une liste.
  * Sinon, indiquer qu’aucune solution n’a été trouvée.

In [ ]:
from constraint import Problem, AllDifferentConstraint

def solve_sudoku_csp_no_sugar(grid):
    # 1. 创建一个问题实例
    problem = Problem()

    # ==========================================
    # 第一步：定义变量 (Variables) 和 值域 (Domains)
    # 遍历 9x9 的每一个格子
    # ==========================================
    for i in range(9):      # i 代表行号 (0-8)
        for j in range(9):  # j 代表列号 (0-8)
            
            # 获取当前格子的数值
            current_val = grid[i][j]
            
            # 这里的逻辑是：
            # 如果格子是 0 (空)，那么它的可能性(Domain)是 1 到 9
            # 如果格子已经有数字 (比如 5)，那么它的可能性就只有 [5]
            domain = []
            if current_val == 0:
                # 相当于 range(1, 10)
                for num in range(1, 10):
                    domain.append(num)
            else:
                domain.app
                end(current_val)
            
            # 向问题中添加变量
            # 变量名我们用坐标元组 (i, j) 来表示，比如 (0, 0) 代表左上角第一个格子
            problem.addVariable((i, j), domain)

    # ==========================================
    # 第二步：添加约束 (Constraints)
    # 核心规则：AllDifferentConstraint (所有值必须不同)
    # ==========================================

    # 2.1 行约束：每一行的 9 个数字必须互不相同
    for i in range(9):
        # 原代码用了列表推导式，这里拆解开：
        # 我们需要收集第 i 行所有格子的坐标
        row_cells = []
        for j in range(9):
            row_cells.append((i, j))
        
        # 添加约束：这一行(row_cells)里的所有变量，取值不能重复
        problem.addConstraint(AllDifferentConstraint(), row_cells)

    # 2.2 列约束：每一列的 9 个数字必须互不相同
    for j in range(9):
        # 收集第 j 列所有格子的坐标
        col_cells = []
        for i in range(9):
            col_cells.append((i, j))
            
        # 添加约束
        problem.addConstraint(AllDifferentConstraint(), col_cells)

    # 2.3 宫格约束 (难点)：每个 3x3 小方块内的数字必须互不相同
    # 数独一共有 9 个 3x3 的宫格
    # br (block row) 代表宫格的起始行：0, 3, 6
    # bc (block col) 代表宫格的起始列：0, 3, 6
    for br in [0, 3, 6]: 
        for bc in [0, 3, 6]:
            
            # 收集当前 3x3 宫格内的 9 个坐标
            block_cells = []
            
            # 在当前宫格起始点 (br, bc) 的基础上，横向纵向各偏移 0, 1, 2
            for di in range(3):      # delta i (行偏移)
                for dj in range(3):  # delta j (列偏移)
                    row = br + di
                    col = bc + dj
                    block_cells.append((row, col))
            
            # 添加约束：这 9 个格子(block_cells)互不相同
            problem.addConstraint(AllDifferentConstraint(), block_cells)

    # ==========================================
    # 第三步：求解并格式化输出
    # ==========================================
    
    # 让库帮我们算，返回的是一个字典列表
    # 格式类似：[{ (0,0): 5, (0,1): 3, ... }, {解法2...}]
    solutions = problem.getSolutions()

    # 如果没找到解
    if len(solutions) == 0:
        print("Aucune solution n'a été trouvée.")
        return []

    # 把库返回的字典格式，转换回我们要的 9x9 二维列表格式
    formatted_grids = []
    
    for sol in solutions:
        # 创建一个 9x9 的全 0 空网格
        solved_grid = []
        for _ in range(9):
            row = [0] * 9 # 创建包含9个0的行
            solved_grid.append(row)
        
        # 遍历字典，把值填进去
        # sol.items() 会给出 ((行, 列), 值)
        for coordinate, value in sol.items():
            row_idx = coordinate[0]
            col_idx = coordinate[1]
            solved_grid[row_idx][col_idx] = value
            
        formatted_grids.append(solved_grid)

    return formatted_grids

### **Exercice 5**
Utilisez votre fonction pour resoudre le jeu suivant.

In [8]:
# Grille de Sudoku (0 représente une case vide)
grid = [[5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]]

In [9]:
res = solve_sudoku_csp(grid)
if res :
    visualize_sudoku(res[0])
else:
    print("aucune solution")

5,3,4,6,7,8,9,1,2
6,7,2,1,9,5,3,4,8
1,9,8,3,4,2,5,6,7
8,5,9,7,6,1,4,2,3
4,2,6,8,5,3,7,9,1
7,1,3,9,2,4,8,5,6
9,6,1,5,3,7,2,8,4
2,8,7,4,1,9,6,3,5
3,4,5,2,8,6,1,7,9


# **Partie 2 : Comparaison CSP vs SAT**

Nous allons comparer les deux solveurs sur 10 sudokus differents. Pour cela, voici 10 grilles.

In [10]:
grid1 = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

In [11]:
grid2 = [
    [0, 0, 3, 0, 2, 0, 6, 0, 0],
    [9, 0, 0, 3, 0, 5, 0, 0, 1],
    [0, 0, 1, 8, 0, 6, 4, 0, 0],
    [0, 0, 8, 1, 0, 2, 9, 0, 0],
    [7, 0, 0, 0, 0, 0, 0, 0, 8],
    [0, 0, 6, 7, 0, 8, 2, 0, 0],
    [0, 0, 2, 6, 0, 9, 5, 0, 0],
    [8, 0, 0, 2, 0, 3, 0, 0, 9],
    [0, 0, 5, 0, 1, 0, 3, 0, 0]
]

In [12]:
grid3 = [
    [0, 2, 0, 6, 0, 8, 0, 0, 0],
    [5, 8, 0, 0, 0, 9, 7, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 0],
    [3, 7, 0, 0, 0, 0, 5, 0, 0],
    [6, 0, 0, 0, 0, 0, 0, 0, 4],
    [0, 0, 8, 0, 0, 0, 0, 1, 3],
    [0, 0, 0, 0, 2, 0, 0, 0, 0],
    [0, 0, 9, 8, 0, 0, 0, 3, 6],
    [0, 0, 0, 3, 0, 6, 0, 9, 0]
]

In [13]:
grid4 = [
    [0, 0, 5, 3, 0, 0, 0, 0, 0],
    [8, 0, 0, 0, 0, 0, 0, 2, 0],
    [0, 7, 0, 0, 1, 0, 5, 0, 0],
    [4, 0, 0, 0, 0, 5, 3, 0, 0],
    [0, 1, 0, 0, 7, 0, 0, 0, 6],
    [0, 0, 3, 2, 0, 0, 0, 8, 0],
    [0, 6, 0, 5, 0, 0, 0, 0, 9],
    [0, 0, 4, 0, 0, 0, 0, 3, 0],
    [0, 0, 0, 0, 0, 9, 7, 0, 0]
]

In [14]:
grid5 = [
    [8, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 3, 6, 0, 0, 0, 0, 0],
    [0, 7, 0, 0, 9, 0, 2, 0, 0],
    [0, 5, 0, 0, 0, 7, 0, 0, 0],
    [0, 0, 0, 0, 4, 5, 7, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 3, 0],
    [0, 0, 1, 0, 0, 0, 0, 6, 8],
    [0, 0, 8, 5, 0, 0, 0, 1, 0],
    [0, 9, 0, 0, 0, 0, 4, 0, 0]
]

In [15]:
grid6 = [
    [0, 0, 0, 6, 0, 0, 4, 0, 0],
    [7, 0, 0, 0, 0, 3, 6, 0, 0],
    [0, 0, 0, 0, 9, 1, 0, 8, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 5, 0, 1, 8, 0, 0, 0, 3],
    [0, 0, 0, 3, 0, 6, 0, 4, 5],
    [0, 4, 0, 2, 0, 0, 0, 6, 0],
    [9, 0, 3, 0, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 1, 0, 0]
]

In [16]:
grid7 = [
    [0, 2, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 6, 0, 0, 0, 0, 3],
    [0, 7, 4, 0, 8, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 0, 2],
    [0, 8, 0, 0, 4, 0, 0, 1, 0],
    [6, 0, 0, 5, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 7, 8, 0],
    [5, 0, 0, 0, 0, 9, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 4, 0]
]

In [17]:
grid8 = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 8, 5],
    [0, 0, 1, 0, 2, 0, 0, 0, 0],
    [0, 0, 0, 5, 0, 7, 0, 0, 0],
    [0, 0, 4, 0, 0, 0, 1, 0, 0],
    [0, 9, 0, 0, 0, 0, 0, 0, 0],
    [5, 0, 0, 0, 0, 0, 0, 7, 3],
    [0, 0, 2, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 9]
]

In [18]:
grid9 = [
    [0, 0, 0, 2, 6, 0, 7, 0, 1],
    [6, 8, 0, 0, 7, 0, 0, 9, 0],
    [1, 9, 0, 0, 0, 4, 5, 0, 0],
    [8, 2, 0, 1, 0, 0, 0, 4, 0],
    [0, 0, 4, 6, 0, 2, 9, 0, 0],
    [0, 5, 0, 0, 0, 3, 0, 2, 8],
    [0, 0, 9, 3, 0, 0, 0, 7, 4],
    [0, 4, 0, 0, 5, 0, 0, 3, 6],
    [7, 0, 3, 0, 1, 8, 0, 0, 0]
]

In [19]:
grid10 = [
    [0, 2, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 6, 0, 0, 0, 0, 3],
    [0, 7, 4, 0, 8, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 0, 2],
    [0, 8, 0, 0, 4, 0, 0, 1, 0],
    [6, 0, 0, 5, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 7, 8, 0],
    [5, 0, 0, 0, 0, 9, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 4, 0]
]

### **Exercice 6**

Créez une fonction comparer_solveurs qui prend comme entrée une liste de grilles, résout tous les jeux avec les deux solveurs, enregistre le temps d'exécution à chaque fois, et retourne les résultats sous forme de DataFrame.

Utilisez cette fonction pour résoudre tous les jeux et affichez ensuite les résultats.

In [ ]:
import time
import pandas as pd
import copy
from tabulate import tabulate

def comparer_solveurs(grids):
    """
    比较 SAT 和 CSP 求解器在给定数独网格列表上的性能。
    Compare les performances des solveurs SAT et CSP sur une liste de grilles.
    """
    results = []
    
    for i, original_grid in enumerate(grids):
        # 使用深拷贝以确保每个求解器都使用干净的初始网格
        # Utiliser deepcopy pour s'assurer que chaque solveur utilise une grille propre
        grid_sat = copy.deepcopy(original_grid)
        grid_csp = copy.deepcopy(original_grid)
        
        # --- SAT Solver ---
        start_sat = time.time()
        solve_sudoku_sat(grid_sat) # 确保 solve_sudoku_sat 已在前面定义
        end_sat = time.time()
        time_sat = end_sat - start_sat
        
        # --- CSP Solver ---
        start_csp = time.time()
        solve_sudoku_csp(grid_csp) # 确保 solve_sudoku_csp 已在前面定义
        end_csp = time.time()
        time_csp = end_csp - start_csp
        
        # 记录结果 / Enregistrer les résultats
        results.append({
            "Sudoku": f"Grid {i+1}",
            "SAT Time (s)": time_sat,
            "CSP Time (s)": time_csp,
            "Diff (CSP-SAT)": time_csp - time_sat
        })
    
    return pd.DataFrame(results)

# 1. 准备网格列表 (确保 grid1 到 grid10 已经在之前的单元格中运行并定义)
# Préparer la liste des grilles
all_grids = [grid1, grid2, grid3, grid4, grid5, grid6, grid7, grid8, grid9, grid10]

# 2. 运行比较函数
# Exécuter la fonction de comparaison
df_results = comparer_solveurs(all_grids)


+---+---------+----------------------+----------------------+-----------------------+
|   | Sudoku  |     SAT Time (s)     |     CSP Time (s)     |    Diff (CSP-SAT)     |
+---+---------+----------------------+----------------------+-----------------------+
| 0 | Grid 1  | 0.022109031677246094 | 0.007368564605712891 | -0.014740467071533203 |
| 1 | Grid 2  | 0.015587091445922852 | 0.00578618049621582  | -0.009800910949707031 |
| 2 | Grid 3  |  0.0855262279510498  | 0.19257616996765137  |  0.10704994201660156  |
| 3 | Grid 4  | 0.40662121772766113  | 0.09663796424865723  |  -0.3099832534790039  |
| 4 | Grid 5  |  0.5544865131378174  |  1.211951732635498   |  0.6574652194976807   |
| 5 | Grid 6  |  0.3968641757965088  |  0.5412707328796387  |  0.14440655708312988  |
| 6 | Grid 7  |  0.400815486907959   |  0.9678118228912354  |  0.5669963359832764   |
| 7 | Grid 8  |  0.4848513603210449  |  4.663164854049683   |   4.178313493728638   |
| 8 | Grid 9  | 0.011567831039428711 | 0.0052821636199

In [21]:
from tabulate import tabulate
print(tabulate(df_results, headers = 'keys', tablefmt = 'pretty'))

+---+---------+----------------------+----------------------+-----------------------+
|   | Sudoku  |     SAT Time (s)     |     CSP Time (s)     |    Diff (CSP-SAT)     |
+---+---------+----------------------+----------------------+-----------------------+
| 0 | Grid 1  | 0.022109031677246094 | 0.007368564605712891 | -0.014740467071533203 |
| 1 | Grid 2  | 0.015587091445922852 | 0.00578618049621582  | -0.009800910949707031 |
| 2 | Grid 3  |  0.0855262279510498  | 0.19257616996765137  |  0.10704994201660156  |
| 3 | Grid 4  | 0.40662121772766113  | 0.09663796424865723  |  -0.3099832534790039  |
| 4 | Grid 5  |  0.5544865131378174  |  1.211951732635498   |  0.6574652194976807   |
| 5 | Grid 6  |  0.3968641757965088  |  0.5412707328796387  |  0.14440655708312988  |
| 6 | Grid 7  |  0.400815486907959   |  0.9678118228912354  |  0.5669963359832764   |
| 7 | Grid 8  |  0.4848513603210449  |  4.663164854049683   |   4.178313493728638   |
| 8 | Grid 9  | 0.011567831039428711 | 0.0052821636199